In [ ]:
### Imports go here
import tweepy
from tweepy import OAuthHandler
import pandas as pd
import json
import time
import datetime
import pytz
from google.colab import files
import statistics


consumer_key = "Wj2lTBcBUscgL8kYfDp0wtft4"
consumer_secret = "0TJPWVSnSCd0bvMUmWRxe9FrlEerx3b4VZ0i2PgMjQ0YCDJFhd"
access_token = "1643093376491999233-i2CjqnMkx1G9fnMVpoGpPDUbAfR0sC"
access_token_secret = "gRX8fwR2qD4Yo19PHRLfPaLKVpbBGBqlqpvBZlfN8iDXl"

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
### Function to retrieve tweet from brand's stream
engagementrate = []
subs = [280931, 280919, 280935, 280949, 280999]
date = [datetime.datetime(2023,6,6,0,0,0,0,pytz.UTC),datetime.datetime(2023,6,7,0,0,0,0,pytz.UTC),datetime.datetime(2023,6,8,0,0,0,0,pytz.UTC)
        ,datetime.datetime(2023,6,9,0,0,0,0,pytz.UTC),datetime.datetime(2023,6,10,0,0,0,0,pytz.UTC)]
amplification = []
likespertweet = []
user_id = 'omegawatches'

def getTweet(x):
  tweets = []
  id = ''
  text = ''
  retweet = 0
  favorite = 0
  #user = api.get_user(screen_name=user_id)
  tweet = api.user_timeline(screen_name = user_id, count=100)
  tweetsondate = []
  i = 0

  for t in tweet:
                       # datetime.datetime.now()
      if t.created_at <= date[x]:
        if i < 30:
           # calculate engagement for each tweet             #user.followers_count
          engagement = (t.retweet_count + t.favorite_count) / subs[x] * 100
          # calculate amplification for each tweet
          amplification = (t.retweet_count) / subs[x] * 100
                                                                                             #user.followers_count
          tweetsondate.append([t.id, t.text, t.created_at, t.retweet_count, t.favorite_count, subs[x], engagement, amplification])
          i+=1
  columns= ["Tweet ID", "Tweet Text", "Tweet date", "Retweet Count", "Favorite Count", "Followers", "Engagement Rate", "Amplification Rate"]
  tweets_df = pd.DataFrame(tweetsondate, columns = columns)
  return tweets_df

for x in range(len(date)):
  engagement =getTweet(x)
  total = totalampli = 0
  for i in range(len(engagement)):
    total += engagement['Engagement Rate'][i]
    totalampli += engagement['Amplification Rate'][i]

  #calculate average likes per tweet
  likespertweet.append(statistics.mean(engagement['Favorite Count']))

  #calculation engagement rate
  engagementrate.append(total/len(engagement))

  #calculate amplication rate
  amplification.append(totalampli/len(engagement))

In [ ]:
tweets = [4219, 4220, 4220, 4221, 4223]
avgtweets = []
for x in range(len(date)):
  user = api.get_user(screen_name=user_id)
              # datetime.datetime.now()
  difference = date[x] - user.created_at
  #calculate average tweets per day
  avgtweets.append(tweets[x]/difference.days)

In [ ]:
columns= ["Date", "Followers", "Audience Growth Rate", "Engagement Rate", "Amplification Rate", "Average Likes per tweet", "Average Tweets per day"]
tweets_df = pd.DataFrame(columns = columns)
tweets_df['Followers'] = subs
tweets_df['Engagement Rate'] = engagementrate
tweets_df['Date'] = date
tweets_df['Amplification Rate'] = amplification
tweets_df['Average Likes per tweet'] = likespertweet
tweets_df['Average Tweets per day'] = avgtweets

In [ ]:
#calculation of audience growth rate
tweets_df['Audience Growth Rate'][0] = 0
for i in range(len(tweets_df)-1):
  tweets_df['Audience Growth Rate'][i+1] = (tweets_df['Followers'][i+1] - tweets_df['Followers'][i]) / tweets_df['Followers'][i]

<ipython-input-5-ec5894ed59b2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['Audience Growth Rate'][0] = 0
<ipython-input-5-ec5894ed59b2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df['Audience Growth Rate'][i+1] = (tweets_df['Followers'][i+1] - tweets_df['Followers'][i]) / tweets_df['Followers'][i]


In [ ]:
tweets_df

,Date,Followers,Audience Growth Rate,Engagement Rate,Amplification Rate,Average Likes per tweet,Average Tweets per day
0,2023-06-06 00:00:00+00:00,280931,0,0.129474,0.011616,331.100000,0.952370
1,2023-06-07 00:00:00+00:00,280919,-0.000043,0.128198,0.011593,327.566667,0.952381
2,2023-06-08 00:00:00+00:00,280935,0.000057,0.128191,0.011592,327.566667,0.952166
3,2023-06-09 00:00:00+00:00,280949,0.00005,0.126630,0.011485,323.500000,0.952177
4,2023-06-10 00:00:00+00:00,280999,0.000178,0.127925,0.012159,325.300000,0.952413


In [ ]:
tweets_df.to_csv('omega_twitter.csv', index = False)